<a href="https://colab.research.google.com/github/ribesstefano/ml4justice-feature-predictor/blob/main/notebooks/ml4justice_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install lightning -qqq

In [2]:
import pandas as pd

base_dir = "../data/processed/"
dataset_df = pd.read_csv(base_dir + 'dataset-sentenze-omicidio-stradale-with-summaries-translated.csv', encoding='utf-8')

dataset_df = dataset_df.drop_duplicates(subset=['raw text'])

# Fix an empy 'facts' column...
dataset_df['facts'] = dataset_df.apply(lambda x: x['facts'] if pd.notna(x['facts']) else x['raw text'], axis=1)
dataset_df[dataset_df['facts'].isna()]

print(len(dataset_df))

79


In [3]:
label_columns = dataset_df.iloc[:, 2:46].columns.tolist()
id2label = {i: l for i, l in enumerate(label_columns)}
label2id = {v: k for k, v in id2label.items()}
id2label

{0: 'localita commerciale o luogo di lavoro',
 1: 'vittima eta avanzata',
 2: 'vittima eta giovane',
 3: 'lavori stradali di manutenzione',
 4: 'guida senza patente valida imputato',
 5: 'guida senza patente valida vittima',
 6: 'piu imputati',
 7: 'strada a doppio senso di marcia',
 8: 'prossimita di un incrocio o curva',
 9: 'ostacolo prevedibile dalla vittima',
 10: 'ostacolo prevedibile dall imputato',
 11: 'ostacolo evitabile dall imputato',
 12: 'eccesso verso imputato',
 13: 'eccesso verso vittima',
 14: 'neopat imputato',
 15: 'veicoli non abilitato a circolare o senza assicurazione vittima',
 16: 'veicoli non abilitato a circolare o senza assicurazione imputato',
 17: 'invasione corsia opposta imputato',
 18: 'vittima ubriaca o drogata',
 19: 'imputato ubriaco o drogato',
 20: 'giovane eta imputato',
 21: 'giorno',
 22: 'notte',
 23: 'segnaletica non visibile o assente',
 24: 'maltempo',
 25: 'violazione stop o semaforo o codice della strada da parte della vittima',
 26: 'viol

## PyTorch Dataset

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, vectorizer=None, precomputed_features=False):
        self.precomputed_features = precomputed_features
        if precomputed_features:
            self.features = texts
        else:
            if vectorizer:
                self.vectorizer = vectorizer
                self.features = self.vectorizer.transform(texts)
            else:
                self.vectorizer = CountVectorizer(
                    ngram_range=(1, 2),  # For example, unigrams and bigrams
                    stop_words=['italian'],
                    binary=True,
                    decode_error='ignore',
                )
                self.features = self.vectorizer.fit_transform(texts)
        self.labels = labels
        if self.features.shape[0] != len(self.labels):
            raise ValueError(
                f"Number of samples in features ({self.features.shape[0]}) "
                f"and labels ({len(self.labels)}) do not match."
            )

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if self.precomputed_features:
            return {
                'X': torch.tensor(self.features[idx]).squeeze().float(),
                'y': torch.tensor(self.labels.iloc[idx]).float(),
            }
        else:
            return {
                'X': torch.tensor(self.features[idx].toarray()).squeeze().float(),
                'y': torch.tensor(self.labels.iloc[idx]).float(),
            }

# Data Preparation
texts = dataset_df['raw text']
labels = dataset_df.iloc[:, 2:46]  # Adjust indices as needed
dataset = TextDataset(texts, labels)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)
# dataset[0]
batch = next(iter(train_loader))
print(batch['X'].size())
print(batch['y'].size())

torch.Size([8, 102139])
torch.Size([8, 44])


### Data Balance

SMOTE is not supported for multilabel classification. We will use a custom sampler to balance the dataset.

In [5]:
# from imblearn.over_sampling import SMOTE

# sm = SMOTE(random_state=42)
# X_res, y_res = sm.fit_resample(vectors, dataset_df.iloc[:, 2:46])

TODO: An idea would be to translate back and forth the text in order to generate variations. Still a work in progress.

In [6]:
# !pip install deep-translator -qqq

import textwrap
from deep_translator import GoogleTranslator

translator_it2zh = GoogleTranslator(source='it', target='zh-CN')
translator_zh2it = GoogleTranslator(source='zh-CN', target='it')

def translate_back_and_forth(text, translator1=translator_it2zh, translator2=translator_zh2it):
    return translator2.translate(translator1.translate(text)[:4999])

test_it = "All'imputato era stata contestata una colpa generica e la violazione dell'art. 141 C.d.S. poiché alla guida della sua autovettura, nel percorrere la strada provinciale n. (Omissis), alla guida dell'autocarro (Omissis), aveva tenuto una velocità non adeguata e comunque pari a 80 Km/h non commisurata allo stato dei luoghi,in considerazione delle condizioni metereologiche e dell'ora serale, e, affrontando un tratto di strada curvilineo, aveva perso il controllo del veicolo che, capovolgendosi, rovinava contro un manufatto, così cagionando la morte per gravissimo trauma cranico del F.C., seduto al lato passeggero anteriore.  La Corte territoriale, nel respingere i motivi di gravame, riteneva esaustiva la motivazione della sentenza di prime cure in ordine alle modalità del sinistro, su cui si soffermava rispondendo alle osservazioni mosse dall'appellante; confermava la statuizione del primo giudice in ordine al mancato riconoscimento delle circostanze attenuanti generiche, la durata della sanzione amministrativa accessoria e l'ammontare delle spese liquidate dal Tribunale in favore delle parti civili.  2. Avverso tale pronuncia ha proposto ricorso l'imputato, a mezzo del difensore di fiducia, per tre distinti motivi.  2.1. Con un primo motivo lamenta vizio di mancanza della motivazione in ordine alla ricostruzione della dinamica del sinistro, non essendo illustrate le ragioni per cui la Corte territoriale aveva deciso di disattendere le risultanze della consulenza tecnica di parte, puntualmente illustrate negli specifici motivi di appello.  La sentenza impugnata risultava priva dei passaggi logici ed argomentativi indispensabili, essendosi limitata al mero richiamo delle considerazioni svolte dal giudice di primo grado, tacendo del tutto le ragioni per le quali erano stati ritenuti inattendibili le fonti di prova a discarico dell'imputato. "
# print(textwrap.fill(test_it, 150))
# print('-' * 150)
# print(textwrap.fill(translate_back_and_forth(test_it + test_it), 150))
# print('-' * 150)

In [7]:
translate_back_and_forth(test_it + test_it)

"L'imputato è stato accusato di negligenza grave e violazione dell'art. 141 C.d.S. perché stava percorrendo una strada provinciale. Il conducente del camion (Omissis) ha mantenuto una velocità inadeguata, comunque pari a 80 km/h, sproporzionata rispetto alle condizioni del luogo, tenuto conto delle condizioni atmosferiche e dell'ora della notte, ed affrontando la situazione su un percorso tortuoso tratto di strada investito dal veicolo, il veicolo ha perso il controllo, si è ribaltato e si è schiantato contro un edificio provocando la morte di F.C, che era seduto sul lato del passeggero anteriore, per gravi ferite alla testa. Nel respingere i motivi di ricorso, la Corte territoriale ha ritenuto esaustiva la motivazione della decisione di primo grado rispetto alla natura dell'incidente e si è quindi concentrata su questo punto nel rispondere alle conclusioni del ricorrente, confermando la decisione del primo giudice di non riconoscere le attenuanti generali, il termine per le sanzioni a

In [8]:
from tqdm import tqdm

if 'mid_lang_zh' not in dataset_df.columns:
    tqdm.pandas(desc="Translation progress")
    dataset_df['mid_lang_zh'] = dataset_df['facts'].progress_apply(lambda x: translate_back_and_forth(x[:4999]))

In [9]:
translator_it2en = GoogleTranslator(source='it', target='en')
translator_en2it = GoogleTranslator(source='en', target='it')

if 'mid_lang_en' not in dataset_df.columns:
    tqdm.pandas(desc="Translation progress")
    dataset_df['mid_lang_en'] = dataset_df['facts'].progress_apply(lambda x: translate_back_and_forth(x[:4999], translator_it2en, translator_en2it))

In [10]:
dataset_df.to_csv(base_dir + 'dataset-sentenze-omicidio-stradale-with-summaries-translated.csv', encoding='utf-8', index=False)

In [11]:
def balance_train_df(train_df, label_columns=label_columns, oversampling_dumping=1.0, min_oversampling=2):
    """ Balance the train_df by oversampling the minority class for each label in label_columns """
    balanced_dfs = []

    for label in label_columns:
        # Get the number of samples with label set to 1 and 0
        label_1_count = train_df[train_df[label] == 1].shape[0]
        label_0_count = train_df[train_df[label] == 0].shape[0]

        # Skip oversampling if all labels are the same
        if label_1_count == 0 or label_0_count == 0:
            balanced_dfs.append(train_df)
            continue

        # Determine the minority and majority class
        minority_class = 1 if label_1_count < label_0_count else 0
        majority_class = 1 if minority_class == 0 else 0

        # Calculate the oversampling factor
        oversampling_factor = label_0_count // label_1_count if minority_class == 1 else label_1_count // label_0_count
        oversampling_factor = max(min_oversampling, int(oversampling_factor * oversampling_dumping))

        # Oversample the minority class
        minority_df = train_df[train_df[label] == minority_class]
        oversampled_minority_df = pd.concat([minority_df] * oversampling_factor)

        # Combine the oversampled minority class with the majority class
        balanced_df = pd.concat([train_df[train_df[label] == majority_class], oversampled_minority_df])

        # # Print the number of samples with label set to 1 and 0
        # print(f"{label}: {len(balanced_df)} (0/1: {balanced_df[balanced_df[label] == 0].shape[0]} / {balanced_df[balanced_df[label] == 1].shape[0]})")

        balanced_dfs.append(balanced_df)

    # Merge the balanced dataframes for each label
    balanced_train_df = pd.concat(balanced_dfs)

    return balanced_train_df

print(len(dataset_df))
print(len(balance_train_df(dataset_df, oversampling_dumping=0.3)))

79
4230


## PyTorch Lightning Model

In [12]:
from torch import nn
import numpy as np
from torchmetrics import (
    MetricCollection,
    Accuracy,
    AUROC,
    Precision,
    Recall,
    F1Score,
    # MeanAbsoluteError,
    # MeanSquaredError,
)
from torchmetrics.functional.classification import (
    accuracy,
    auroc,
    precision,
    recall,
    f1_score,
)

In [23]:
import warnings
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

class TextClassifier(pl.LightningModule):
    def __init__(
            self,
            num_features,
            num_labels,
            hidden_size=0,
            dropout=0.6,
            learning_rate=1e-3,
        ):
        super().__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        if hidden_size:
            self.model = nn.Sequential(
                nn.Linear(num_features, hidden_size),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, num_labels),
            )
        else:
            self.model = nn.Sequential(
                nn.Linear(num_features, num_labels),
            )
        # self.bin_loss = nn.BCELoss()
        self.bin_loss = nn.BCEWithLogitsLoss()
        # Metrics, a separate metrics collection is defined for each stage
        # NOTE: According to the PyTorch Lightning docs, "similar" metrics,
        # i.e., requiring the same computation, should be optimized w/in a
        # metrics collection.
        stages = ['train_metrics', 'val_metrics', 'test_metrics']
        self.metrics = nn.ModuleDict({s: MetricCollection({
            'acc': Accuracy(task='multilabel', num_labels=num_labels), # Default: macro
            'roc_auc': AUROC(task='multilabel', num_labels=num_labels),
            'precision': Precision(task='multilabel', num_labels=num_labels),
            'recall': Recall(task='multilabel', num_labels=num_labels),
            'f1_score': F1Score(task='multilabel', num_labels=num_labels),
            'acc_weighted': Accuracy(task='multilabel', num_labels=num_labels, average='weighted'),
            'roc_auc_weighted': AUROC(task='multilabel', num_labels=num_labels, average='weighted'),
            'precision_weighted': Precision(task='multilabel', num_labels=num_labels, average='weighted'),
            'recall_weighted': Recall(task='multilabel', num_labels=num_labels, average='weighted'),
            'f1_score_weighted': F1Score(task='multilabel', num_labels=num_labels, average='weighted'),
        }, prefix=s.replace('metrics', '')) for s in stages})

    def forward(self, x):
        return self.model(x)  # Sigmoid for binary classification

    def step(self, batch, stage='train'):
        x, y = batch['X'], batch['y']
        preds = self.forward(x)
        loss = self.bin_loss(preds, y)
        preds = nn.Sigmoid()(preds)
        self.metrics[f'{stage}_metrics'].update(preds, y.to(torch.int32))
        self.log(f'{stage}_loss', loss, on_epoch=True, prog_bar=True)
        self.log_dict(self.metrics[f'{stage}_metrics'], on_epoch=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self.step(batch, stage='train')

    def validation_step(self, batch, batch_idx):
        return self.step(batch, stage='val')

    def test_step(self, batch, batch_idx):
        return self.step(batch, stage='test')

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)


In [ ]:

# Data Preparation
texts = dataset_df['raw text'] # dataset_df['facts']
labels = dataset_df.iloc[:, 2:46]  # Adjust indices as needed
# Fit the vectorizer on the entire dataset
vectorizer = CountVectorizer(
    ngram_range=(1, 2),  # For example, unigrams and bigrams
    stop_words=['italian'],
    binary=True,
    # max_df=0.3740124132589796,
    decode_error='ignore',
)
scaler = MaxAbsScaler()

# Data Preparation
BALANCE_TRAIN_SET = True
if BALANCE_TRAIN_SET:
    texts = dataset_df['raw text'].tolist() + dataset_df['facts'].tolist() + dataset_df['mid_lang_zh'].tolist() + dataset_df['mid_lang_en'].tolist()
    print('fitting...')
    vectorizer.fit(texts)

    print('transforming...')
    for text_col in ['raw text', 'facts', 'mid_lang_zh', 'mid_lang_en']:
        dataset_df[f'count_vectors_{text_col}'] = scaler.fit_transform(vectorizer.transform(dataset_df[text_col])).toarray().tolist()

    print('oversampling...')
    oversampled_df = balance_train_df(dataset_df, oversampling_dumping=0.1)

    print('appending...')
    texts = []
    labels = []
    for text_col in ['raw text', 'facts', 'mid_lang_zh', 'mid_lang_en']:
        texts.append(oversampled_df[f'count_vectors_{text_col}'].tolist())
        labels.append(oversampled_df.iloc[:, 2:46])
    print('stacking...')
    texts = np.vstack(texts)
    labels = pd.DataFrame(labels)


    # train_df = []
    # dataset_df['count_vectors'] = scaler.fit_transform(vectorizer.transform(dataset_df['raw text'])).toarray() # .tolist()
    # train_df.append(dataset_df.copy())
    # dataset_df['count_vectors'] = scaler.fit_transform(vectorizer.transform(dataset_df['facts'])).toarray() # .tolist()
    # train_df.append(dataset_df.copy())
    # dataset_df['count_vectors'] = scaler.fit_transform(vectorizer.transform(dataset_df['mid_lang_zh'])).toarray() # .tolist()
    # train_df.append(dataset_df.copy())
    # dataset_df['count_vectors'] = scaler.fit_transform(vectorizer.transform(dataset_df['mid_lang_en'])).toarray() # .tolist()
    # train_df.append(dataset_df.copy())

    # train_df = balance_train_df(dataset_df, oversampling_dumping=0.1)
    # train_df = pd.concat(train_df, axis=0, ignore_index=True)

    # display(train_df)
    # display(train_df)
        
    # --------------------------------
    # vectors = vectorizer.fit_transform(texts)
    # vectors = scaler.fit_transform(vectors)
    # vectors = vectors.toarray().tolist()
    # dataset_df['count_vectors'] = vectors
    # oversampled_df = balance_train_df(dataset_df, oversampling_dumping=0.9)
    # train_df = pd.concat([
    #     dataset_df,
    #     oversampled_df.sample(400, random_state=42),
    # ])
    # texts = np.array(train_df['count_vectors'].tolist())
    # labels = train_df.iloc[:, 2:46]
else:
    vectors = vectorizer.fit_transform(dataset_df['raw text'])
    vectors = scaler.fit_transform(vectors)

    vectors = vectors.toarray().tolist()
    dataset_df['count_vectors'] = vectors

    texts = dataset_df['count_vectors'].to_numpy()
    labels = dataset_df.iloc[:, 2:46]

print(texts.shape)
print(labels.shape)
print(texts)

num_features = len(vectorizer.get_feature_names_out())
num_labels = labels.shape[1]

dataset = TextDataset(texts, labels, vectorizer, precomputed_features=True)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)
next(iter(train_loader))

# # Training
# with warnings.catch_warnings():
#     # Suppress warning messages
#     warnings.filterwarnings("ignore", category=UserWarning)
#     warnings.filterwarnings("ignore", category=FutureWarning)
#     # Model Initialization
#     model = TextClassifier(
#         num_features,
#         num_labels,
#         hidden_size=32,
#         dropout=0.6,
#         learning_rate=5e-4,
#     )
#     # Trainer setup
#     trainer = pl.Trainer(
#         max_epochs=1,
#         # max_steps=200,
#         enable_model_summary=True,
#         precision="32" if torch.cuda.is_available() else "32-true",
#         accelerator="gpu" if torch.cuda.is_available() else "cpu",
#         devices="auto",
#         enable_checkpointing=False,
#         logger=False,
#         enable_progress_bar=True,
#     )
#     trainer.fit(model, train_loader, train_loader)
#     trainer.test(model, train_loader)[0]
#     y_preds = trainer.predict(model, torch.vstack([x['X'] for x in train_loader]))[0].unsqueeze(dim=0)

# y_preds = nn.Sigmoid()(y_preds)
# print(y_preds)
# y_targets = np.zeros(y_preds.shape)
# y_targets[np.where(y_preds) == 1] = 1
# y_targets = torch.Tensor(y_targets).to(torch.int32)

# print(accuracy(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=0.3))
# print(precision(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=0.3))
# print(recall(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=0.3))
# print(f1_score(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=0.3))
# print(auroc(y_preds, y_targets, task='multilabel', num_labels=num_labels))

# # Wrape the vectorizer and scaler in a pipeline
# from sklearn.pipeline import Pipeline

# pipeline = Pipeline([
#     ('vectorizer', vectorizer),
#     ('scaler', scaler),
# ])

# # Save the pipeline to disk
# import joblib

# joblib.dump(pipeline, '../models/pipeline.joblib')

# # Save the trainer to disk
# trainer.save_checkpoint('../models/model.ckpt')


In [ ]:
def load_system(vectorizer_path, model_path):
    # Load the pipeline from disk
    pipeline = joblib.load(vectorizer_path)
    # Load trainer and model from disk
    model = TextClassifier.load_from_checkpoint(model_path)
    # Define new trainer
    trainer = pl.Trainer(
        max_epochs=1,
        # max_steps=200,
        enable_model_summary=True,
        precision="32" if torch.cuda.is_available() else "32-true",
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices="auto",
        enable_checkpointing=False,
        logger=False,
        enable_progress_bar=True,
    )
    return pipeline, model, trainer

pipeline, model, trainer = load_system(
    '../models/pipeline.joblib',
    '../models/model.ckpt',
)

## Leave-One-Out Traning

### Precompute Embeddings

In [13]:
# Load standard scaler from sklearn
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()

In [14]:
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

# Data Preparation
texts = dataset_df['raw text'] # dataset_df['facts']
labels = dataset_df.iloc[:, 2:46]  # Adjust indices as needed
full_dataset = TextDataset(texts, labels)
# Fit the vectorizer on the entire dataset
vectorizer = CountVectorizer(
    ngram_range=(1, 2),  # For example, unigrams and bigrams
    stop_words=['italian'],
    binary=True,
    # max_df=0.3740124132589796,
    decode_error='ignore',
)
scaler = MaxAbsScaler()

vectors = vectorizer.fit_transform(dataset_df['raw text'])
vectors = scaler.fit_transform(vectors)

vectors = vectors.toarray().tolist()
num_features_vec = len(vectorizer.get_feature_names_out())
dataset_df['count_vectors'] = vectors
print(f"{num_features_vec:,}")

102,139


In [ ]:
# !python -m spacy download it_core_news_lg -qqq

In [15]:
num_features_emb = 300

In [ ]:
# import spacy

# nlp = spacy.load('it_core_news_lg')

# # texts = dataset_df['raw text'].tolist()
# texts = dataset_df['facts'].tolist()
# vectors = [doc.vector for doc in nlp.pipe(texts, batch_size=16)]
# num_features_emb = np.array(vectors).shape[1]
# dataset_df['text_embedding'] = vectors

# del nlp

# len(dataset_df['text_embedding'])

### Start Training Loop

In [21]:
# from lightning.pytorch import seed_everything

# # sets seeds for numpy, torch and python.random.
# seed_everything(42, workers=True)

In [25]:
import pytorch_lightning as pl
from sklearn.model_selection import LeaveOneOut
from torch.utils.data import Subset
import warnings

USE_SPACY = False
BALANCE_TRAIN_SET = True

num_features = num_features_emb if USE_SPACY else num_features_vec
features_col = "text_embedding" if USE_SPACY else "count_vectors"

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
report_table = []
report_table_per_threshold = []
report_table_per_label = []
predictions = []
targets = []

for k, (train_index, test_index) in enumerate(loo.split(full_dataset)):
    # Splitting the dataset
    # train_subset = Subset(full_dataset, train_index)
    # test_subset = Subset(full_dataset, test_index)
    train_df = dataset_df.iloc[train_index, :].copy()
    test_df = dataset_df.iloc[test_index, :].copy()
    # Train
    if BALANCE_TRAIN_SET:
        oversampled = balance_train_df(train_df, oversampling_dumping=0.9)
        train_df = pd.concat([
            train_df,
            oversampled.sample(400, random_state=42),
        ])
    texts = np.array(train_df[features_col]) # ['raw text']
    labels = train_df.iloc[:, 2:46]  # Adjust indices as needed
    train_subset = TextDataset(texts, labels, vectorizer, precomputed_features=True)
    # Test
    texts = np.array(test_df[features_col]) # ['raw text']
    labels = test_df.iloc[:, 2:46]  # Adjust indices as needed
    test_subset = TextDataset(texts, labels, vectorizer, precomputed_features=True)

    # Creating data loaders for training and testing
    train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=1, shuffle=False)

    # Model Initialization
    num_labels = labels.shape[1]
    model = TextClassifier(
        num_features,
        num_labels,
        hidden_size=512,
        dropout=0.6,
        learning_rate=5e-4,
    )
    # Trainer setup
    trainer = pl.Trainer(
        max_epochs=1,
        # max_steps=200,
        enable_model_summary=False,
        precision="32" if torch.cuda.is_available() else "32-true",
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices="auto",
        enable_checkpointing=False,
        logger=False,
        enable_progress_bar=False,
    )
    # Train the model
    with warnings.catch_warnings():
        # Suppress warning messages
        warnings.filterwarnings("ignore", category=UserWarning)
        warnings.filterwarnings("ignore", category=FutureWarning)
        trainer.fit(model, train_loader, test_loader)
        # Evaluate the model on the test data
        results = trainer.test(model, test_loader, verbose=False)[0]
        # Store predictions and labels for later processing
        test_elem = next(iter(test_loader))
        y_preds = trainer.predict(model, test_elem['X'])[0].unsqueeze(dim=0).to(torch.float32)
        y_preds = nn.Sigmoid()(y_preds)
        y_targets = test_elem['y'].to(torch.int32)

    predictions.append(y_preds)
    targets.append(y_targets)

    # Display current state metrics
    if k % 10 == 0:
        y_pred = torch.vstack(predictions)
        y_true = torch.vstack(targets)
        score_table = []
        for average in ['macro', 'micro', 'weighted']:
            score_table.append({
                'average': average,
                'test_accuracy': accuracy(y_pred, y_true, task='multilabel', num_labels=num_labels, average=average).item(),
                'test_precision': precision(y_pred, y_true, task='multilabel', num_labels=num_labels, average=average).item(),
                'test_recall': recall(y_pred, y_true, task='multilabel', num_labels=num_labels, average=average).item(),
                'test_f1_score': f1_score(y_pred, y_true, task='multilabel', num_labels=num_labels, average=average).item(),
                'roc_auc': auroc(y_pred, y_true, task='multilabel', num_labels=num_labels, average=average).item(),
            })
        display(pd.DataFrame(score_table))

    results['k'] = k
    report_table.append(results)
    print(f"K = {k} done.")

predictions = torch.vstack(predictions)
targets = torch.vstack(targets)
report_table = pd.DataFrame(report_table)
report_table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


,test_loss,test_acc,test_acc_weighted,test_f1_score,test_f1_score_weighted,test_precision,test_precision_weighted,test_recall,test_recall_weighted,test_roc_auc,test_roc_auc_weighted,k
0,0.524833,0.909091,0.5,0.666667,0.5,1.0,0.5,0.5,0.5,0.0,0.0,0


K = 0 done.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


,test_loss,test_acc,test_acc_weighted,test_f1_score,test_f1_score_weighted,test_precision,test_precision_weighted,test_recall,test_recall_weighted,test_roc_auc,test_roc_auc_weighted,k
0,0.524833,0.909091,0.50,0.666667,0.50,1.0,0.50,0.50,0.50,0.0,0.0,0
1,0.087794,0.977273,0.75,0.857143,0.75,1.0,0.75,0.75,0.75,0.0,0.0,1


...

K = 77 done.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


,test_loss,test_acc,test_acc_weighted,test_f1_score,test_f1_score_weighted,test_precision,test_precision_weighted,test_recall,test_recall_weighted,test_roc_auc,test_roc_auc_weighted,k
0,0.524833,0.909091,0.500000,0.666667,0.500000,1.00,0.500000,0.500000,0.500000,0.0,0.0,0
1,0.087794,0.977273,0.750000,0.857143,0.750000,1.00,0.750000,0.750000,0.750000,0.0,0.0,1
2,0.481476,0.931818,0.500000,0.666667,0.500000,1.00,0.500000,0.500000,0.500000,0.0,0.0,2
3,1.122859,0.909091,0.428571,0.600000,0.428571,1.00,0.428571,0.428571,0.428571,0.0,0.0,3
4,0.676073,0.886364,0.428571,0.545455,0.428571,0.75,0.428571,0.428571,0.428571,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
74,1.153479,0.840909,0.333333,0.461538,0.333333,0.75,0.333333,0.333333,0.333333,0.0,0.0,74
75,0.769632,0.840909,0.300000,0.461538,0.300000,1.00,0.300000,0.300000,0.300000,0.0,0.0,75
76,0.389514,0.886364,0.444444,0.615385,0.444444,1.00,0.444444,0.444444,0.444444,0.0,0.0,76
77,0.032855,0.977273,1.000000,0.857143,1.000000,0.75,1.000000,1.000000,1.000000,0.0,0.0,77


K = 78 done.


,test_loss,test_acc,test_acc_weighted,test_f1_score,test_f1_score_weighted,test_precision,test_precision_weighted,test_recall,test_recall_weighted,test_roc_auc,test_roc_auc_weighted,k
0,0.524833,0.909091,0.500000,0.666667,0.500000,1.00,0.500000,0.500000,0.500000,0.0,0.0,0
1,0.087794,0.977273,0.750000,0.857143,0.750000,1.00,0.750000,0.750000,0.750000,0.0,0.0,1
2,0.481476,0.931818,0.500000,0.666667,0.500000,1.00,0.500000,0.500000,0.500000,0.0,0.0,2
3,1.122859,0.909091,0.428571,0.600000,0.428571,1.00,0.428571,0.428571,0.428571,0.0,0.0,3
4,0.676073,0.886364,0.428571,0.545455,0.428571,0.75,0.428571,0.428571,0.428571,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
74,1.153479,0.840909,0.333333,0.461538,0.333333,0.75,0.333333,0.333333,0.333333,0.0,0.0,74
75,0.769632,0.840909,0.300000,0.461538,0.300000,1.00,0.300000,0.300000,0.300000,0.0,0.0,75
76,0.389514,0.886364,0.444444,0.615385,0.444444,1.00,0.444444,0.444444,0.444444,0.0,0.0,76
77,0.032855,0.977273,1.000000,0.857143,1.000000,0.75,1.000000,1.000000,1.000000,0.0,0.0,77


In [ ]:
report_table.agg(['mean', 'std'])

In [26]:
report_table.to_csv(f"../data/scores-nlp-mlp-loo.csv", index=False)
basedir = f"../data/processed/"
torch.save(predictions, f'{basedir}/nlp-mlp-loo-predictions.pt')
torch.save(targets, f'{basedir}/nlp-mlp-loo-targets.pt')

In [ ]:
def get_predictions(probs, threshold=0.5):
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= threshold)] = 1
    return predictions

def get_predicted_labels(predictions, id2label=id2label):
    # Turn predicted ID's into actual label names
    labeled_predictions = []
    for pred in predictions:
        tmp = [id2label[idx] for idx, label in enumerate(pred) if label == 1.0]
        labeled_predictions.append(tmp)
    return labeled_predictions

# Data Preparation
texts = dataset_df['raw text']
labels = dataset_df.iloc[:, 2:46]  # Adjust indices as needed
dataset = TextDataset(texts, labels)
train_loader = DataLoader(dataset, batch_size=8, shuffle=False)

X = torch.vstack([x['X'] for x in train_loader])
y_logits = torch.vstack(trainer.predict(model, X))
y_probs = nn.Sigmoid()(y_logits).numpy()
print(y_probs.shape)

predictions = get_predictions(y_probs, threshold=0.7)
predicted_labels = get_predicted_labels(predictions, id2label=id2label)
for i, (t, l) in enumerate(zip(texts, predicted_labels)):
    print('-' * 80)
    print(f'Sentenza n. {i}:')
    print('-' * 80)
    print(l)
    print(t[500:1000])
    if i > 1:
        break

#### Evaluate on Test Set

In [ ]:
import json
import re

test_filename = 'test_facts'
test_filename = 'test_texts'

with open(f'../data/processed/{test_filename}.json', 'r') as f:
    test_texts = json.load(f)

vectors = vectorizer.transform(test_texts.values())
vectors = scaler.transform(vectors)
vectors = torch.Tensor(vectors.toarray()).to(torch.float32)
vectors.shape

y_logits = torch.vstack(trainer.predict(model, vectors))
y_probs = nn.Sigmoid()(y_logits).numpy()
print(y_probs.shape)
# Check if the sum of probabilities for each sample is 1
print((np.sum(y_probs, axis=1) == np.ones(y_probs.shape[0]).all()))
print(np.sum(y_probs, axis=1))

predictions = get_predictions(y_probs, threshold=0.5)
predicted_labels = get_predicted_labels(predictions, id2label=id2label)
predicted_labels

table = []
for filename, pred, prob in zip(test_texts.keys(), predictions.tolist(), y_probs.tolist()):
    tmp = {
        'filename': filename.strip('\.\.\\data\\raw\\Sentenze\\test\\'),
        'numero sentenza': re.search(r'\.\.\\data\\raw\\Sentenze\\test\\Cass.\s(\d*)\.pdf', filename).group(1),
    }
    # tmp.update({label: int(p) for label, p in zip(id2label.values(), pred)})
    tmp.update({label: f'{p:.3f}' for label, p in zip(id2label.values(), prob)})
    tmp['labels'] = ', '.join(get_predicted_labels([pred], id2label=id2label)[0])
    table.append(tmp)
table = pd.DataFrame(table)
table.to_csv(f'../data/processed/{test_filename}_predictions.csv', index=False)
table

## Decision Tree Pipeline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

# Load vocabulary from file '../data/raw/660000_parole_italiane.txt' as list
vocabulary = []
with open('../data/raw/660000_parole_italiane.txt', 'r') as f:
    vocabulary = f.read().splitlines()

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 2),
        stop_words=None, # ['italian'], # NOTE: The only supported language is English, apparently
        strip_accents='ascii',
        max_df=0.3,
        # token_pattern=r"(?u)\b\w{4,}\b", # Default: r"(?u)\b\w\w+\b"
        # max_features=10_000,
        # vocabulary=vocabulary,
    )),
    ('scaler', MaxAbsScaler()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier(random_state=42, max_depth=5)))
    # ('clf', DecisionTreeClassifier(random_state=42))
])

In [ ]:
# Data Preparation: Balance the dataset
oversampled = balance_train_df(dataset_df, oversampling_dumping=0.9)
train_df = pd.concat([
    dataset_df,
    oversampled.sample(200, random_state=42),
])

# texts = dataset_df['raw text']
texts = train_df['facts']
labels = train_df.iloc[:, 2:46]  # Adjust indices as needed

# Fit the pipeline to your data
pipeline.fit(texts, labels)

for i, row in dataset_df.sample(5, random_state=43).iterrows():
    pred = pipeline.predict([row['raw text']])[0]
    print(row['facts'])
    print([id2label[idx] for idx, label in enumerate(pred) if label == 1.0])
    print([id2label[idx] for idx, label in enumerate(row.iloc[2:46]) if label == 1.0])
    print('-' * 80)

### Optuna-Based Hyperparameter Tuning

In [ ]:
import warnings
from sklearn.metrics import accuracy_score
from optuna import create_study, Trial, visualization

def get_pipeline(**kwargs):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            ngram_range=kwargs.get('ngram_range', (1, 2)),
            stop_words=None, # ['italian'], # NOTE: The only supported language is English, apparently
            strip_accents='ascii',
            max_df=kwargs.get('max_df', 0.7),
            token_pattern=kwargs.get('token_pattern', r"(?u)\b\w\w+\b"),
            # max_features=10_000,
            # vocabulary=vocabulary,
        )),
        ('scaler', MaxAbsScaler()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier(random_state=42, max_depth=kwargs.get('max_depth', 5), criterion=kwargs.get('criterion', 'gini'))))
    ])
    return pipeline

def objective(trial: Trial):
    # Define the pipeline
    pipeline_params = {
        'ngram_range': trial.suggest_categorical('ngram_range', [(1, 1), (1, 2)]),
        'max_df': trial.suggest_float('max_df', 0.1, 0.6),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'token_pattern': trial.suggest_categorical('token_pattern', [r"(?u)\b\w{4,}\b", r"(?u)\b\w\w+\b"]),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss']),
    }
    pipeline = get_pipeline(**pipeline_params)

    # Data Preparation: Balance the dataset
    oversampled = balance_train_df(dataset_df)
    train_df = pd.concat([
        dataset_df,
        oversampled.sample(500, random_state=42),
    ])

    texts = train_df['facts']
    labels = train_df.iloc[:, 2:46].to_numpy() 

    # Fit the pipeline to your data
    pipeline.fit(texts, labels)

    # Evaluate the pipeline
    texts = dataset_df['raw text']
    labels = dataset_df.iloc[:, 2:46].to_numpy()
    
    y_preds = pipeline.predict(texts)
    y_targets = labels
    full_accuracy = accuracy_score(y_targets, y_preds)

    # Calculate the accuracy for each label
    accuracies = []
    for label_id, _ in id2label.items():
        accuracies.append(accuracy_score(y_targets[:, label_id], y_preds[:, label_id]))
    per_label_accuracy = np.array(accuracies).mean()
    return full_accuracy + per_label_accuracy


study = create_study(direction='maximize')

# Suppress warning messages
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    study.optimize(objective, n_trials=10)
# Print best trial params
print(study.best_trial.params)

In [ ]:
import matplotlib.pyplot as plt

visualization.matplotlib.plot_optimization_history(study)
plt.show()
visualization.matplotlib.plot_param_importances(study)
plt.show()

### Leave-One-Out Traning

In [ ]:
import torch
from torch import nn
import numpy as np
from torchmetrics import (
    MetricCollection,
    Accuracy,
    AUROC,
    Precision,
    Recall,
    F1Score,
    # MeanAbsoluteError,
    # MeanSquaredError,
)
from torchmetrics.functional.classification import (
    accuracy,
    auroc,
    precision,
    recall,
    f1_score,
)

In [ ]:
from sklearn.model_selection import LeaveOneOut

BALANCE_TRAIN_SET = True

features_col = "raw text"

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
report_table = []
predictions = []
targets = []

for k, (train_index, test_index) in enumerate(loo.split(dataset_df)):
    # Splitting the dataset
    train_df = dataset_df.iloc[train_index, :].copy()
    test_df = dataset_df.iloc[test_index, :].copy()
    # Train
    if BALANCE_TRAIN_SET:
        oversampled = balance_train_df(train_df, oversampling_dumping=0.9)
        train_df = pd.concat([
            train_df,
            oversampled.sample(400, random_state=42),
        ])
    X_train = np.array(train_df[features_col])
    y_train = np.array(train_df.iloc[:, 2:46])
    # Test
    X_test = np.array(test_df[features_col])
    y_test = np.array(test_df.iloc[:, 2:46])
    # Get pipeline
    pipeline = get_pipeline(**study.best_trial.params)
    # Fit the pipeline to your data
    pipeline.fit(X_train, y_train)
    # Evaluate the pipeline
    y_preds = torch.Tensor(np.array(pipeline.predict(X_test)))
    y_targets = torch.Tensor(y_test).to(torch.int32)
    # Metrics
    num_labels = y_train.shape[1]
    metrics = MetricCollection({
            'acc': Accuracy(task='multilabel', num_labels=num_labels), # Default: macro
            'roc_auc': AUROC(task='multilabel', num_labels=num_labels),
            'precision': Precision(task='multilabel', num_labels=num_labels),
            'recall': Recall(task='multilabel', num_labels=num_labels),
            'f1_score': F1Score(task='multilabel', num_labels=num_labels),
            'acc_weighted': Accuracy(task='multilabel', num_labels=num_labels, average='weighted'),
            'roc_auc_weighted': AUROC(task='multilabel', num_labels=num_labels, average='weighted'),
            'precision_weighted': Precision(task='multilabel', num_labels=num_labels, average='weighted'),
            'recall_weighted': Recall(task='multilabel', num_labels=num_labels, average='weighted'),
            'f1_score_weighted': F1Score(task='multilabel', num_labels=num_labels, average='weighted'),
        }, prefix='test_')
    # Suppress warning messages
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        warnings.filterwarnings("ignore", category=FutureWarning)
        metrics.update(y_preds, y_targets)
        results = {m: val.item() if isinstance(val, torch.Tensor) else val for m, val in metrics.compute().items()}
        results['k'] = k
    # Store predictions and labels for later processing
    predictions.append(y_preds)
    targets.append(y_targets)
    report_table.append(results)
    display(pd.DataFrame(report_table).round(3))
    print(f"K = {k} done.")

predictions = torch.vstack(predictions)
targets = torch.vstack(targets)
report_table = pd.DataFrame(report_table)
report_table

In [ ]:
report_table.agg(['mean', 'std'])

In [ ]:
report_table.to_csv(f"../data/scores-nlp-decision_tree-loo.csv", index=False)
basedir = f"../data/processed/"
torch.save(predictions, f'{basedir}/nlp-decision_tree-loo-predictions.pt')
torch.save(targets, f'{basedir}/nlp-decision_tree-loo-targets.pt')

### Explainability And Tree Plotting

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

def tree_decision_path(pipeline, single_text, label='notte', path_as_string=True):
    # Transform the text to the tfidf representation
    tfidf_vector = pipeline.named_steps['tfidf'].transform([single_text])
    
    # Get the decision tree classifier
    if hasattr(pipeline.named_steps['clf'], 'estimators_'):
        decision_tree = pipeline.named_steps['clf'].estimators_[label2id[label]]
    else:
        decision_tree = pipeline.named_steps['clf']
    
    # Get the feature names
    feature_names = pipeline.named_steps['tfidf'].get_feature_names_out()
    
    # Get the decision path in the tree for the tfidf vector
    node_indicator = decision_tree.decision_path(tfidf_vector)
    leaf_id = decision_tree.apply(tfidf_vector)
    
    # Output decision path
    feature_path = []
    for node_id in node_indicator.indices[node_indicator.indptr[0]:node_indicator.indptr[1]]:
        if leaf_id[0] == node_id:
            continue  # Skip leaf node

        # Check if value of the feature is greater than threshold
        if (tfidf_vector[0, decision_tree.tree_.feature[node_id]] > decision_tree.tree_.threshold[node_id]):
            threshold_sign = ">"
        else:
            threshold_sign = "<="

        # Append the feature and threshold
        if path_as_string:
            feat_in = feature_names[decision_tree.tree_.feature[node_id]]
            threshold = decision_tree.tree_.threshold[node_id]
            feature_path.append(f"{feat_in} {threshold_sign} {threshold:.4f}")
        else:
            feature_path.append((
                feature_names[decision_tree.tree_.feature[node_id]],
                threshold_sign,
                decision_tree.tree_.threshold[node_id]
            ))

    return feature_path

# Example usage
for i, row in dataset_df.sample(5, random_state=43).iterrows():
    # text_example = row['raw text']
    text_example = row['facts']
    y_true = row.iloc[2:46].tolist()
    y_pred = pipeline.predict([text_example])[0]

    print(row['facts'])
    # print([id2label[idx] for idx, label in enumerate(y_pred) if label == 1.0])
    # print([id2label[idx] for idx, label in enumerate(y_true) if label == 1.0])
    # Calculate accuracy
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")

    for idx, (label_true, label_pred) in enumerate(zip(y_true, y_pred)):
        if label_true == 0 and label_pred == 0:
            continue
        label = id2label[idx]
        path = tree_decision_path(pipeline, text_example, label=label)
        print(f'\t* Path for label "{label}": {"" if label_true == label_pred else "(INCORRECT)"}')
        for step, node in enumerate(path):
            print(f"\t\t{step + 1}. {node}")
    print('-' * 80)

In [ ]:
# Given a label, plot the corresponding decision tree in the pipeline
from sklearn import tree
import matplotlib.pyplot as plt
import re
import matplotlib

def replace_text(obj):
    if type(obj) == matplotlib.text.Annotation:
        txt = obj.get_text()
        # txt = re.sub("\nsamples[^$]*class", "\nclass", txt)
        txt = re.sub("\nvalue[^$]*", "", txt)
        obj.set_text(txt)
    return obj

def plot_decision_tree(pipeline, label='notte', plot_values=True):
    # Get the decision tree classifier
    # Get the decision tree classifier
    if hasattr(pipeline.named_steps['clf'], 'estimators_'):
        decision_tree = pipeline.named_steps['clf'].estimators_[label2id[label]]
    else:
        decision_tree = pipeline.named_steps['clf']

    # Get the feature names
    feature_names = list(pipeline.named_steps['tfidf'].get_feature_names_out())

    # Plot the tree
    with warnings.catch_warnings():
        # Suppress warning messages
        warnings.filterwarnings("ignore", category=UserWarning)
        warnings.filterwarnings("ignore", category=FutureWarning)
        fig, ax = plt.subplots(figsize=(20, 20))
        tree.plot_tree(decision_tree, feature_names=feature_names, filled=True, ax=ax,)
        if not plot_values:
            ax.properties()['children'] = [replace_text(i) for i in ax.properties()['children']]
        plt.savefig(f'../data/figures/decision_tree_{label}.pdf', bbox_inches='tight')
        fig.show()

plot_decision_tree(pipeline, label='notte')
plot_decision_tree(pipeline, label='giovane eta imputato')
plot_decision_tree(pipeline, label='violazione stop o semaforo o codice della strada da parte imputato')

## Evaluation

In [82]:
import pandas as pd
import torch
from torch import nn
import numpy as np
from torchmetrics import (
    MetricCollection,
    Accuracy,
    AUROC,
    Precision,
    Recall,
    F1Score,
    # MeanAbsoluteError,
    # MeanSquaredError,
)
from torchmetrics.functional.classification import (
    accuracy,
    auroc,
    precision,
    recall,
    f1_score,
)

### Boolean Function and Decision Scores

In [99]:
def determine_response(FO, AO, DV, VV, defendant_violations=None, victim_violations=None):
    """
    Determine the response based on the given parameters.

    Parameters:
    FO (int): Foreseeable Obstacle (0 or 1)
    AO (int): Avoidable Obstacle (0 or 1)
    DV (int): Defendant's Violation (0 or 1)
    VV (int): Victim's Violation (0 or 1)
    defendant_violations (int, optional): Number of violations by the defendant
    victim_violations (int, optional): Number of violations by the victim

    Returns:
    int: The response (0 or 1) based on the given parameters
    """

    # Scenarios with Response = -
    if FO == 0 and AO == 1:
        return None  # Not a reasonable scenario

    # Scenarios with Response = *
    if DV == 1 and VV == 1:
        if defendant_violations is not None and victim_violations is not None:
            return 1 if defendant_violations > victim_violations else 0
        else:
            # Without the number of violations, we cannot determine the response
            return None

    # Other scenarios 
    if FO == 1 and DV == 1:
        return 1
    else:
        return 0

# Example usage
response = determine_response(1, 0, 1, 1, defendant_violations=3, victim_violations=2)
response

1

In [100]:
defendant_features = [
    'guida senza patente valida imputato',
    'ostacolo prevedibile dall imputato',
    'ostacolo evitabile dall imputato',
    'eccesso verso imputato',
    'veicoli non abilitato a circolare o senza assicurazione imputato',
    'invasione corsia opposta imputato',
    'imputato ubriaco o drogato',
    'violazione stop o semaforo o codice della strada da parte imputato',
    'guida brusca o manovra brusca imputato',
    'uso telefono alla guida',
]
victim_features = [
    'guida senza patente valida vittima',
    'ostacolo prevedibile dalla vittima',
    'eccesso verso vittima',
    'veicoli non abilitato a circolare o senza assicurazione vittima',
    'vittima ubriaca o drogata',
    'violazione stop o semaforo o codice della strada da parte della vittima',
    'vittima priva di giubbino e bretelle riflettenti',
    'guida brusca o manovra brusca della vittima',
    'mancanza cintura di sicurezza o casco vittima',
    'attraversamento vittima senza strisce o sbucata improvvisamente',
    'attraversamento corretto della vittima',
]

def get_predictions(probs, threshold=0.5):
    pred = np.zeros(probs.shape, dtype=np.int32)
    pred[np.where(probs >= threshold)] = 1
    return pred

# Get the index of the label in defendant_features
defendant_features_idx = np.array([label2id[label] for label in defendant_features])
victim_features_idx = np.array([label2id[label] for label in victim_features])
idx_feat = np.concatenate([defendant_features_idx, victim_features_idx])
num_labels_feat = idx.size

def get_decision(pred):
    # Get the predictions for the defendant and victim features
    defendant_features = pred[defendant_features_idx]
    victim_features = pred[victim_features_idx]
    return determine_response(
        FO=pred[label2id['ostacolo prevedibile dall imputato']],
        AO=pred[label2id['ostacolo evitabile dall imputato']],
        DV=defendant_features.any(),
        VV=victim_features.any(),
        defendant_violations=defendant_features.sum(),
        victim_violations=victim_features.sum(),
    )

# Example usage on random predictions
pred = np.random.randint(0, 2, size=num_labels)
print(get_decision(pred))

1


### Get All Metrics and Save Them

In [107]:
for feature_extractor in ['mlp', 'decision_tree']:
    basedir = f"../data/processed/"
    predictions = torch.load(f'{basedir}/nlp-{feature_extractor}-loo-predictions.pt')
    targets = torch.load(f'{basedir}/nlp-{feature_extractor}-loo-targets.pt')
    num_labels = targets.size()[-1]

    print(f"Feature Extractor: {feature_extractor}")
    # Calculate scores
    score_table = []
    for average in ['macro', 'micro', 'weighted']:
        score_table.append({
            'average': average,
            # Get scores for all labels
            'accuracy': accuracy(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
            'precision': precision(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
            'recall': recall(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
            'f1_score': f1_score(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
            'roc_auc': auroc(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
            # Get scores for only the defendant and victim features
            'feat_accuracy': accuracy(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
            'feat_precision': precision(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
            'feat_recall': recall(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
            'feat_f1_score': f1_score(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
            'feat_roc_auc': auroc(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        })
    score_table = pd.DataFrame(score_table)
    score_table.to_csv(f"../data/nlp-{feature_extractor}-loo-scores.csv", index=False)
    display(score_table)

    # Calculate final ruling decision scores 
    decision_pred = []
    decision_true = []
    for y_pred, y_true in zip(predictions.numpy(), targets.numpy()):
        decision_pred.append(get_decision(y_pred))
        decision_true.append(get_decision(y_true))
    # Replace None with 0, i.e., non-convinced by default, and get torch Tensor
    decision_pred = np.array(decision_pred)
    decision_pred[np.where(decision_pred == None)] = 0
    decision_pred = decision_pred.astype(np.int32)
    decision_pred = torch.Tensor(decision_pred).to(torch.int32)
    # Get torch Tensor from the true decisions
    decision_true = torch.Tensor(decision_true).to(torch.int32)
    # Store metrics to file
    scores = [{
        'Feature Extractor': 'MLP' if feature_extractor == 'mlp' else 'Decision Tree',
        'Accuracy': accuracy(decision_pred, decision_true, task='binary').item(),
        'F1 Score': f1_score(decision_pred, decision_true, task='binary').item(),
        'ROC-AUC': auroc(decision_pred.to(torch.float32), decision_true, task='binary').item(),
        'Precision': precision(decision_pred, decision_true, task='binary').item(),
        'Recall': recall(decision_pred, decision_true, task='binary').item(),
    }]
    scores = pd.DataFrame(scores)
    scores.to_csv(f"../data/nlp-{feature_extractor}-loo-decision_scores.csv", index=False)
    print("Final Ruling Decision Scores:")
    display(scores)

    # Calculate scores per label
    report_table_per_label = []
    for label_id, label_name in id2label.items():
        y_preds = predictions[:, label_id]
        y_targets = targets[:, label_id]
        threshold = 0.5
        tmp = {
            'label': label_name,
            'test_accuracy': accuracy(y_preds, y_targets, task='binary', threshold=threshold).item(),
            'test_precision': precision(y_preds, y_targets, task='binary', threshold=threshold).item(),
            'test_recall': recall(y_preds, y_targets, task='binary', threshold=threshold).item(),
            'test_f1_score': f1_score(y_preds, y_targets, task='binary', threshold=threshold).item(),
            'support': y_targets.sum().item(),
        }
        report_table_per_label.append(tmp)
    report_table_per_label = pd.DataFrame(report_table_per_label)
    report_table_per_label.to_csv(f"../data/nlp-{feature_extractor}-loo-scores_per_label.csv", index=False)
    print("Scores per label:")
    display(report_table_per_label)

    # Calculate scores per threshold if the model is a MLP
    if feature_extractor == 'mlp':
        report_table_per_threshold = []
        for k, (y_preds, y_targets) in enumerate(zip(predictions, targets)):
            y_preds = y_preds.unsqueeze(dim=0)
            y_targets = y_targets.unsqueeze(dim=0)
            for threshold in [0.1 * i for i in range(11)]:
                for average in ['macro', 'micro', 'weighted']:
                    tmp = {
                        'k': k,
                        'threshold': threshold,
                        'average': average,
                        'accuracy': accuracy(y_preds, y_targets, task='multilabel', num_labels=num_labels, average=average, threshold=threshold).item(),
                        'precision': precision(y_preds, y_targets, task='multilabel', num_labels=num_labels, average=average, threshold=threshold).item(),
                        'recall': recall(y_preds, y_targets, task='multilabel', num_labels=num_labels, average=average, threshold=threshold).item(),
                        'f1_score': f1_score(y_preds, y_targets, task='multilabel', num_labels=num_labels, average=average, threshold=threshold).item(),
                    }
                report_table_per_threshold.append(tmp)
        report_table_per_threshold = pd.DataFrame(report_table_per_threshold)
        report_table_per_threshold.to_csv(f"../data/nlp-mlp-loo-scores_per_threshold.csv", index=False)
        print("Scores per threshold:")
        display(report_table_per_threshold)
    print("-" * 80)

Feature Extractor: mlp


c:\Users\ste\Anaconda2\envs\env-thesis\Lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


,average,accuracy,precision,recall,f1_score,roc_auc,feat_accuracy,feat_precision,feat_recall,feat_f1_score,feat_roc_auc
0,macro,0.877445,0.125789,0.086169,0.084720,0.405546,0.875226,0.144510,0.166231,0.152258,0.362268
1,micro,0.877445,0.725166,0.389680,0.506944,0.788690,0.875226,0.740351,0.613372,0.670906,0.848526
2,weighted,0.779562,0.429647,0.389680,0.365656,0.484745,0.778886,0.515296,0.613372,0.556507,0.499126


Final Ruling Decision Scores:


,Feature Extractor,Accuracy,F1 Score,ROC-AUC,Precision,Recall
0,MLP,0.78481,0.879433,0.5,0.78481,1.0


...

Final Ruling Decision Scores:


,Feature Extractor,Accuracy,F1 Score,ROC-AUC,Precision,Recall
0,Decision Tree,0.696203,0.809524,0.528937,0.796875,0.822581


Scores per label:


,label,test_accuracy,test_precision,test_recall,test_f1_score,support
0,localita commerciale o luogo di lavoro,0.810127,0.000000,0.000000,0.000000,8
1,vittima eta avanzata,0.759494,0.083333,0.111111,0.095238,9
2,vittima eta giovane,0.924051,0.000000,0.000000,0.000000,4
3,lavori stradali di manutenzione,0.962025,0.000000,0.000000,0.000000,1
4,guida senza patente valida imputato,0.886076,0.000000,0.000000,0.000000,5
5,guida senza patente valida vittima,1.000000,0.000000,0.000000,0.000000,0
6,piu imputati,0.987342,0.000000,0.000000,0.000000,1
7,strada a doppio senso di marcia,0.607595,0.354839,0.500000,0.415094,22
8,prossimita di un incrocio o curva,0.632911,0.451613,0.538462,0.491228,26
9,ostacolo prevedibile dalla vittima,0.949367,0.000000,0.000000,0.000000,2


--------------------------------------------------------------------------------


### MLP

In [ ]:
basedir = f"../data/processed/"
predictions = torch.load(f'{basedir}/nlp-mlp-loo-predictions.pt')
targets = torch.load(f'{basedir}/nlp-mlp-loo-targets.pt')
predictions.size()

torch.Size([79, 44])

In [ ]:
score_table = []

for average in ['macro', 'micro', 'weighted']:
    score_table.append({
        'average': average,
        'test_accuracy': accuracy(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'test_precision': precision(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'test_recall': recall(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'test_f1_score': f1_score(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'roc_auc': auroc(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
    })
score_table = pd.DataFrame(score_table)
score_table.to_csv(f"../data/nlp-mlp-loo-scores.csv", index=False)
score_table

c:\Users\ste\Anaconda2\envs\env-thesis\Lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


,average,test_accuracy,test_precision,test_recall,test_f1_score,roc_auc
0,macro,0.877445,0.125789,0.086169,0.084720,0.405546
1,micro,0.877445,0.725166,0.389680,0.506944,0.788690
2,weighted,0.779562,0.429647,0.389680,0.365656,0.484745


#### Per-Label and Threshold Analysis

In [ ]:
num_labels = targets.size()[-1]

report_table_per_threshold = []
for k, (y_preds, y_targets) in enumerate(zip(predictions, targets)):
    y_preds = y_preds.unsqueeze(dim=0)
    y_targets = y_targets.unsqueeze(dim=0)
    for threshold in [0.1 * i for i in range(11)]:
        tmp = {
            'k': k,
            'threshold': threshold,
            f'test_accuracy': accuracy(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=threshold).item(),
            f'test_precision': precision(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=threshold).item(),
            f'test_recall': recall(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=threshold).item(),
            f'test_f1_score': f1_score(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=threshold).item(),
            f'test_accuracy_weighted': accuracy(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=threshold, average='weighted').item(),
            f'test_precision_weighted': precision(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=threshold, average='weighted').item(),
            f'test_recall_weighted': recall(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=threshold, average='weighted').item(),
            f'test_f1_score_weighted': f1_score(y_preds, y_targets, task='multilabel', num_labels=num_labels, threshold=threshold, average='weighted').item(),
        }
        report_table_per_threshold.append(tmp)
report_table_per_threshold = pd.DataFrame(report_table_per_threshold)
report_table_per_threshold.to_csv(f"../data/scores-nlp-mlp-loo-per_threshold.csv", index=False)
report_table_per_threshold.groupby('threshold').agg(['mean', 'std'])

In [ ]:
report_table_per_label = []

for label_id, label_name in id2label.items():
    y_preds = predictions[:, label_id]
    y_targets = targets[:, label_id]
    support = y_targets.sum().item()
    for threshold in [0.1 * i for i in range(11)]:
        tmp = {
            'label': label_name,
            'threshold': threshold,
            'test_accuracy': accuracy(y_preds, y_targets, task='binary', threshold=threshold).item(),
            'test_precision': precision(y_preds, y_targets, task='binary', threshold=threshold).item(),
            'test_recall': recall(y_preds, y_targets, task='binary', threshold=threshold).item(),
            'test_f1_score': f1_score(y_preds, y_targets, task='binary', threshold=threshold).item(),
            'support': support,
        }
        tmp.update({
            'test_accuracy_weighted': 0.0 if support == 0 else accuracy(y_preds, y_targets, task='binary', threshold=threshold).item() / support,
            'test_precision_weighted': 0.0 if support == 0 else precision(y_preds, y_targets, task='binary', threshold=threshold).item() / support,
            'test_recall_weighted': 0.0 if support == 0 else recall(y_preds, y_targets, task='binary', threshold=threshold).item() / support,
            'test_f1_score_weighted': 0.0 if support == 0 else f1_score(y_preds, y_targets, task='binary', threshold=threshold).item() / support,
        })
        report_table_per_label.append(tmp)

report_table_per_label = pd.DataFrame(report_table_per_label)
report_table_per_label.to_csv(f"../data/scores-nlp-mlp-loo-per_label.csv", index=False)
report_table_per_label.groupby(['label', 'threshold']).agg(['mean'])

### Decision-Tree

In [ ]:
basedir = f"../data/processed/"
predictions = torch.load(f'{basedir}/nlp-decision_tree-loo-predictions.pt')
targets = torch.load(f'{basedir}/nlp-decision_tree-loo-targets.pt')
predictions.size()

torch.Size([79, 44])

In [ ]:
# score_table = []

# for average in ['macro', 'micro', 'weighted']:
#     score_table.append({
#         'average': average,
#         'test_accuracy': accuracy(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
#         'test_precision': precision(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
#         'test_recall': recall(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
#         'test_f1_score': f1_score(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
#         'roc_auc': auroc(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
#     })
# score_table = pd.DataFrame(score_table)
# score_table.to_csv(f"../data/scores-nlp-decision_tree-loo-scores.csv", index=False)
# score_table


score_table = []
for average in ['macro', 'micro', 'weighted']:
    score_table.append({
        'average': average,
        'accuracy': accuracy(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'precision': precision(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'recall': recall(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'f1_score': f1_score(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'roc_auc': auroc(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),

        'feat_accuracy': accuracy(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        'feat_precision': precision(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        'feat_recall': recall(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        'feat_f1_score': f1_score(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        'feat_roc_auc': auroc(predictions[:, idx_feat], targets[:, idx_feat], task='multilabel', num_labels=num_labels_feat, average=average).item(),
    })
score_table = pd.DataFrame(score_table)
score_table.to_csv(f"../data/nlp-decision_tree-loo-scores.csv", index=False)
score_table

c:\Users\ste\Anaconda2\envs\env-thesis\Lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


,average,accuracy,precision,recall,f1_score,roc_auc,feat_accuracy,feat_precision,feat_recall,feat_f1_score,feat_roc_auc
0,macro,0.832278,0.202319,0.215812,0.206927,0.519429,0.827607,0.248950,0.261071,0.252111,0.506575
1,micro,0.832278,0.482645,0.519573,0.500428,0.706080,0.827607,0.579670,0.613372,0.596045,0.748511
2,weighted,0.718861,0.496505,0.519573,0.505381,0.561307,0.703157,0.603958,0.613372,0.606101,0.546075


In [ ]:
report_table_per_label = []

for label_id, label_name in id2label.items():
    y_preds = predictions[:, label_id]
    y_targets = targets[:, label_id]
    threshold = 0.5
    tmp = {
        'label': label_name,
        'test_accuracy': accuracy(y_preds, y_targets, task='binary', threshold=threshold).item(),
        'test_precision': precision(y_preds, y_targets, task='binary', threshold=threshold).item(),
        'test_recall': recall(y_preds, y_targets, task='binary', threshold=threshold).item(),
        'test_f1_score': f1_score(y_preds, y_targets, task='binary', threshold=threshold).item(),
        'support': y_targets.sum().item(),
    }
    report_table_per_label.append(tmp)
report_table_per_label = pd.DataFrame(report_table_per_label)
report_table_per_label.to_csv(f"../data/scores-nlp-decision_tree-loo-per_label.csv", index=False)
report_table_per_label
# report_table_per_label.groupby(['label']).agg(['mean'])

In [ ]:
decision_pred = []
decision_true = []
for y_pred, y_true in zip(predictions.numpy(), targets.numpy()):
    decision_pred.append(get_decision(y_pred))
    decision_true.append(get_decision(y_true))

# Replace None with 0, i.e., non-convinced by default
decision_pred = np.array(decision_pred)
decision_pred[np.where(decision_pred == None)] = 0
decision_pred = decision_pred.astype(np.int32)
decision_pred = torch.Tensor(decision_pred).to(torch.int32)

decision_true = torch.Tensor(decision_true).to(torch.int32)

scores = [{
    'Accuracy': accuracy(decision_pred, decision_true, task='binary').item(),
    'F1 Score': f1_score(decision_pred, decision_true, task='binary').item(),
    'ROC-AUC': auroc(decision_pred.to(torch.float32), decision_true, task='binary').item(),
    'Precision': precision(decision_pred, decision_true, task='binary').item(),
    'Recall': recall(decision_pred, decision_true, task='binary').item(),
}]
scores = pd.DataFrame(scores)
scores.to_csv(f"../data/nlp-mlp-loo-decision_scores.csv", index=False)
scores

,Accuracy,F1 Score,ROC-AUC,Precision,Recall
0,0.78481,0.879433,0.5,0.78481,1.0


In [ ]:
idx = np.concatenate([defendant_features_idx, victim_features_idx])
num_labels_feat = idx.size

score_table = []
for average in ['macro', 'micro', 'weighted']:
    score_table.append({
        'average': average,
        'accuracy': accuracy(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'precision': precision(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'recall': recall(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'f1_score': f1_score(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),
        'roc_auc': auroc(predictions, targets, task='multilabel', num_labels=num_labels, average=average).item(),

        'feat_accuracy': accuracy(predictions[:, idx], targets[:, idx], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        'feat_precision': precision(predictions[:, idx], targets[:, idx], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        'feat_recall': recall(predictions[:, idx], targets[:, idx], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        'feat_f1_score': f1_score(predictions[:, idx], targets[:, idx], task='multilabel', num_labels=num_labels_feat, average=average).item(),
        'feat_roc_auc': auroc(predictions[:, idx], targets[:, idx], task='multilabel', num_labels=num_labels_feat, average=average).item(),
    })
score_table = pd.DataFrame(score_table)
score_table.to_csv(f"../data/nlp-mlp-loo-scores_relevant_features.csv", index=False)
score_table

c:\Users\ste\Anaconda2\envs\env-thesis\Lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


,average,accuracy,precision,recall,f1_score,roc_auc,feat_accuracy,feat_precision,feat_recall,feat_f1_score,feat_roc_auc
0,macro,0.877445,0.125789,0.086169,0.084720,0.405546,0.875226,0.144510,0.166231,0.152258,0.362268
1,micro,0.877445,0.725166,0.389680,0.506944,0.788690,0.875226,0.740351,0.613372,0.670906,0.848526
2,weighted,0.779562,0.429647,0.389680,0.365656,0.484745,0.778886,0.515296,0.613372,0.556507,0.499126


## Plotting

In [45]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

mlp_report = pd.read_csv('../data/scores-nlp-mlp-loo-scores.csv')
dt_report = pd.read_csv('../data/scores-nlp-decision_tree-loo-scores.csv')
mlp_report['Feature Extractor'] = 'MLP'
dt_report['Feature Extractor'] = 'Decision Tree'
report = pd.concat([mlp_report, dt_report])

report = report.rename(columns={
    'average': 'Reduction type',
    'accuracy': 'Accuracy',
    'f1_score': 'F1 Score',
    'precision': 'Precision',
    'recall': 'Recall',
    'roc_auc': 'ROC-AUC',
    'feat_accuracy': 'Accuracy (DV & VV)',
    'feat_f1_score': 'F1 Score (DV & VV)',
    'feat_precision': 'Precision (DV & VV)',
    'feat_recall': 'Recall (DV & VV)',
    'feat_roc_auc': 'ROC-AUC (DV & VV)',
})
# Set the order of the columns
report = report[[
    'Feature Extractor',
    'Reduction type',
    'Accuracy',
    'F1 Score',
    'ROC-AUC',
    'Precision',
    'Recall',
    'Accuracy (DV & VV)',
    'F1 Score (DV & VV)',
    'Precision (DV & VV)',
    'Recall (DV & VV)',
    'ROC-AUC (DV & VV)',
]]
# Print to Latex table
print(report.to_latex(index=False, float_format="%.3f"))

\begin{tabular}{llrrrrrrrrrr}
\toprule
Feature Extractor & Reduction type &  Accuracy &  F1 Score &  ROC-AUC &  Precision &  Recall &  Accuracy (DV \& VV) &  F1 Score (DV \& VV) &  Precision (DV \& VV) &  Recall (DV \& VV) &  ROC-AUC (DV \& VV) \\
\midrule
              MLP &          macro &     0.877 &     0.085 &    0.406 &      0.126 &   0.086 &               0.875 &               0.152 &                0.145 &             0.166 &              0.362 \\
              MLP &          micro &     0.877 &     0.507 &    0.789 &      0.725 &   0.390 &               0.875 &               0.671 &                0.740 &             0.613 &              0.849 \\
              MLP &       weighted &     0.780 &     0.366 &    0.485 &      0.430 &   0.390 &               0.779 &               0.557 &                0.515 &             0.613 &              0.499 \\
    Decision Tree &          macro &     0.832 &     0.207 &    0.519 &      0.202 &   0.216 &               0.828 &               

C:\Users\ste\AppData\Local\Temp\ipykernel_15488\2380181173.py:40: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(report.to_latex(index=False, float_format="%.3f"))


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

mlp_report = pd.read_csv('../data/scores-nlp-mlp-loo.csv')
dt_report = pd.read_csv('../data/scores-nlp-decision_tree-loo.csv')
mlp_report = mlp_report.agg(['mean']).drop(columns=['test_loss', 'k']).T
dt_report = dt_report.agg(['mean']).drop(columns=['k']).T
mlp_report['Feature Extractor'] = 'MLP'
dt_report['Feature Extractor'] = 'Decision Tree'
report = pd.concat([mlp_report, dt_report])
# Rename index to 'Metric'
report = report.reset_index().rename(columns={'index': 'Metric', 'mean': 'Mean'})
# Map metric names to more readable names
report['Metric'] = report['Metric'].map({
    'test_acc': 'Accuracy',
    'test_acc_weighted': 'Accuracy (weighted)',
    'test_f1_score': 'F1 Score',
    'test_f1_score_weighted': 'F1 Score (weighted)',
    'test_precision': 'Precision',
    'test_precision_weighted': 'Precision (weighted)',
    'test_recall': 'Recall',
    'test_recall_weighted': 'Recall (weighted)',
    'test_roc_auc': 'ROC AUC',
    'test_roc_auc_weighted': 'ROC AUC (weighted)',
})
report

In [ ]:
# Drop AUROC metrics
tmp = report[~report['Metric'].str.contains('ROC AUC')]
# Plot
ax = sns.barplot(data=tmp, x='Metric', hue='Feature Extractor', y='Mean', palette='Blues')
plt.xticks(rotation=90)
# Put the legend flattened on top of the plot
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=2)
# Put values on top of bars, orient them vertically
for p in plt.gca().patches:
    plt.gca().annotate(
        f"{p.get_height():.1%}",
        (p.get_x() + p.get_width() / 2., p.get_height() + 0.05),
        ha='center',
        va='center',
        xytext=(0, 10),
        textcoords='offset points',
        rotation=90,
    )
# Rename axis labels
plt.xlabel('')
plt.ylabel('')
plt.ylim(0, 1.1)
plt.grid(axis='y', alpha=0.5)
plt.show()

In [ ]:
ds = pd.read_csv(f"../data/scores-nlp-decision_tree-loo-per_label.csv")
mlp = pd.read_csv(f"../data/scores-nlp-mlp-loo-per_label.csv")
ds['Feature Extractor'] = 'Decision Tree'
mlp['Feature Extractor'] = 'MLP'
report_table_per_label = pd.concat([ds, mlp])
report_table_per_label.drop(columns=['label']).groupby(['Feature Extractor']).agg(['mean'])

## Stratified K-Fold

Cannot work on multi-labels, skipping.

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import MultiLabelBinarizer

# # Data Preparation
# texts = dataset_df['raw text'] # dataset_df['facts']
# labels = dataset_df.iloc[:, 2:46]  # Adjust indices as needed
# full_dataset = TextDataset(texts, labels)

# mlb = MultiLabelBinarizer()
# mlb.fit(labels_columns)
# print(len(mlb.transform(labels)))

# skf = StratifiedKFold(n_splits=5)

# for k, (train_index, test_index) in enumerate(skf.split(texts.to_numpy(), mlb.transform(labels).T)):
#     # Splitting the dataset
#     train_subset = Subset(full_dataset, train_index)
#     test_subset = Subset(full_dataset, test_index)
#     # Creating data loaders for training and testing
#     train_loader = DataLoader(train_subset, batch_size=8, shuffle=True)
#     test_loader = DataLoader(test_subset, batch_size=1, shuffle=False)
#     # Model Initialization
#     num_features = len(full_dataset.vectorizer.get_feature_names_out())
#     num_labels = labels.shape[1]
#     model = TextClassifier(num_features, num_labels)
#     # Trainer setup
#     trainer = pl.Trainer(
#         max_epochs=5,
#         enable_model_summary=False,
#         precision="32-true",
#     )
#     # Train the model
#     trainer.fit(model, train_loader)
#     # Evaluate the model on the test data
#     results = trainer.test(model, test_loader, verbose=False)[0]
#     results['k'] = k
#     report_table.append(results)
#     print(f"K = {k} done.")

# report_table = pd.DataFrame(report_table)
# report_table.to_csv(f"/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/scores-nlp-count_vectorizer-skf.csv", index=False)
# report_table

In [ ]:
# tmp = report_table.drop(columns=['test_hp_metric', 'k', 'test_opt_score']).describe().round(3)
# tmp.to_csv(f"/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/scores-nlp-count_vectorizer-skf-aggregated.csv")
# tmp